In [10]:
# Upload packages
import numpy as np
import pandas as pd

In [11]:
CD_META = pd.DataFrame(pd.read_csv('~/Desktop/CD.csv'))
UC_META = pd.DataFrame(pd.read_csv('~/Desktop/UC.csv'))
nonIBD_META = pd.DataFrame(pd.read_csv('~/Desktop/nonIBD.csv'))
CD_MGX = pd.read_csv('~/Desktop/Metaphlan4_MGX_CD.csv').set_index('Abundance').T
UC_MGX = pd.read_csv('~/Desktop/Metaphlan4_MGX_UC.csv').set_index('Abundance').T
Control_MGX = pd.read_csv('~/Desktop/Metaphlan4_MGX_Control.csv').set_index('Abundance').T
CD_MTX = pd.read_csv('~/Desktop/Metaphlan4_MTX_CD.csv').set_index('Abundance').T
UC_MTX = pd.read_csv('~/Desktop/Metaphlan4_MTX_UC.csv').set_index('Abundance').T
Control_MTX = pd.read_csv('~/Desktop/Metaphlan4_MTX_Control.csv').set_index('Abundance').T

In [23]:
METADADA = pd.concat([CD_META,UC_META,nonIBD_META],axis=0).set_index(['External ID'])
print(f'METADATA:{METADADA.shape}')
METADADA = METADADA.sort_index()
METADADA.head()

METADATA:(5533, 248)


,data_type,week_num,date_of_receipt,interval_days,visit_num,reads_raw,reads_filtered,reads_qc_fail,reads_human,reads_ribosomal,...,1. Feeling of fatigue,2. Delay or cancel a social engagement,3. Difficulty doing leisure or sports activities,4. How often during the last 2 weeks have you been troubled by pain in the abdom,5. How often during the last two weeks have you felt depressed or discouraged? P,6. Problem with passing large amounts of gas,7. Problem with weight,SIBDQ Score,fecalcal,smoking status
External ID,,,,,,,,,,,,,,,,,,,,,
206454,serology,11.0,11/02/2015,NaN,28.0,58261992.22,13352955.83,9.132710e+06,5.299776e+06,1.043143e+07,...,NaN,NaN,NaN,A good bit of the time,None of the time,Some trouble,NaN,NaN,191.374831,Never smoked
206458,serology,35.0,11/02/2015,NaN,28.0,10312109.08,12085314.83,NaN,5.311762e+06,2.051365e+06,...,A good bit of the time,NaN,A little difficulty,All of the time,None of the time,A little trouble,Some trouble,43.0,377.076602,Current smoker
206459,serology,37.0,11/02/2015,20.0,30.0,37971380.81,23165287.17,4.554554e+07,1.071061e+07,1.138104e+07,...,Most of the time,None of the time,NaN,NaN,None of the time,Some trouble,Some trouble,69.0,423.177321,NaN
206460,serology,39.0,11/02/2015,31.0,NaN,35335551.49,5220834.01,4.691475e+07,5.357971e+06,2.272684e+06,...,None of the time,None of the time,No difficulty,NaN,A little of the time,NaN,No trouble,44.0,76.323999,Never smoked
206461,serology,35.0,11/02/2015,74.0,9.0,5090542.24,13605883.05,1.153700e+07,9.004954e+06,2.740390e+06,...,Most of the time,None of the time,NaN,Most of the time,None of the time,No trouble,No trouble,NaN,379.387126,Never smoked


In [24]:
METADADA = METADADA.loc[METADADA.data_type.isin(['metatranscriptomics','metagenomics']),:]
METADADA.shape

(2473, 248)

In [14]:
def division_species(MGX_file, MTX_file):
    MGX_file = MGX_file.T.sort_index()
    MTX_file = MTX_file.T.sort_index()
    MGX_file = MGX_file.loc[:,MTX_file.columns]
    division_tables = MTX_file.div(MGX_file)
    division_tables.replace([np.inf, -np.inf, np.nan, -np.nan], 0, inplace=True)
    SAVE_TEXONOMY = 's__'
    division_tables = division_tables.T.loc[:,division_tables.T.columns.str.startswith(SAVE_TEXONOMY)]
    return division_tables

In [15]:
MGX_MTX_CD = division_species(CD_MGX,CD_MTX)
MGX_MTX_UC = division_species(UC_MGX,UC_MTX)
MGX_MTX_Control = division_species(Control_MGX,Control_MTX)

In [16]:
for column_name in MGX_MTX_Control.columns:
        column = MGX_MTX_Control[column_name]
        # Get the count of Zeros in column
        count = (column == 0).sum()
        print('Count of zeros in column ', column_name, ' is : ', count)

Count of zeros in column  s__Acidaminococcaceae_bacterium  is :  189
Count of zeros in column  s__Acidaminococcus_intestini  is :  176
Count of zeros in column  s__Actinobaculum_sp_oral_taxon_183  is :  189
Count of zeros in column  s__Actinomyces_SGB17154  is :  189
Count of zeros in column  s__Actinomyces_graevenitzii  is :  189
Count of zeros in column  s__Adlercreutzia_equolifaciens  is :  188
Count of zeros in column  s__Agathobaculum_butyriciproducens  is :  177
Count of zeros in column  s__Agathobaculum_sp_Marseille_P7918  is :  189
Count of zeros in column  s__Aggregatibacter_aphrophilus  is :  189
Count of zeros in column  s__Aggregatibacter_segnis  is :  189
Count of zeros in column  s__Akkermansia_muciniphila  is :  139
Count of zeros in column  s__Akkermansia_sp_KLE1605  is :  180
Count of zeros in column  s__Alistipes_communis  is :  125
Count of zeros in column  s__Alistipes_dispar  is :  173
Count of zeros in column  s__Alistipes_finegoldii  is :  101
Count of zeros in c

In [17]:
MGX_MTX_CD['diagnosis'] = 'CD'
MGX_MTX_UC['diagnosis'] = 'UC'
MGX_MTX_Control['diagnosis'] = 'nonIBD'

In [18]:
MGX_MTX = pd.concat([MGX_MTX_CD,MGX_MTX_UC,MGX_MTX_Control])
MGX_MTX.replace([ np.nan, -np.nan], 0, inplace=True)
print(MGX_MTX.shape)
for column_name in MGX_MTX.columns:
        column = MGX_MTX[column_name]
        # Get the count of Zeros in column
        count = (column == 0).sum()
        print('Count of zeros in column ', column_name, ' is : ', count)

(742, 1220)
Count of zeros in column  s__Abiotrophia_sp_HMSC24B09  is :  742
Count of zeros in column  s__Acidaminococcaceae_unclassified_SGB5785  is :  737
Count of zeros in column  s__Acidaminococcus_intestini  is :  686
Count of zeros in column  s__Acidaminococcus_massiliensis  is :  741
Count of zeros in column  s__Actinobaculum_sp_oral_taxon_183  is :  742
Count of zeros in column  s__Actinomyces_SGB17168  is :  742
Count of zeros in column  s__Actinomyces_dentalis  is :  742
Count of zeros in column  s__Actinomyces_sp_ICM47  is :  742
Count of zeros in column  s__Actinomyces_urogenitalis  is :  741
Count of zeros in column  s__Adlercreutzia_equolifaciens  is :  739
Count of zeros in column  s__Agathobaculum_butyriciproducens  is :  691
Count of zeros in column  s__Aggregatibacter_segnis  is :  740
Count of zeros in column  s__Aggregatibacter_sp_oral_taxon_458  is :  742
Count of zeros in column  s__Akkermansia_muciniphila  is :  603
Count of zeros in column  s__Akkermansia_sp_BIO

In [21]:
MGX_MTX.replace(0,np.nan, inplace=True)
MGX_MTX = MGX_MTX.loc[:,MGX_MTX.isnull().mean() < .7]
MGX_MTX.shape

(742, 26)

In [25]:
METADADA_MGX_MTX = METADADA.merge(MGX_MTX, left_index= True, right_index=True, how='outer')
METADADA_MGX_MTX.to_csv('~/Desktop/Flat_file.csv')